# Nouvelle section

# Nouvelle section

importation et preparation des donnée:

la dataset SANAD_SUBJECT contenant 3 dataset respectivement dupliquer entr train et test data


In [13]:
print("File path:", zip_file_path)
print("File extension:", os.path.splitext(zip_file_path)[-1])


File path: /content/SANAD_SUBSET.zip
File extension: .zip


In [14]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import zipfile

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilisation de l'appareil : {device}")

zip_file_path = "/content/SANAD_SUBSET.zip"
extract_dir = "/content/SANAD_SUBSET"  # Dossier où les données seront extraites

# Extraire le fichier ZIP
if not os.path.exists(extract_dir):
    print("Extraction des fichiers...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
print("Extraction terminée.")

# Fonction pour charger les données
def load_data(data_dir):
    data = {"train": [], "test": []}

    for source in os.listdir(data_dir):
        source_path = os.path.join(data_dir, source)
        for split in ["Train", "Test"]:
            split_path = os.path.join(source_path, split)
            for category in os.listdir(split_path):
                category_path = os.path.join(split_path, category)
                for file in os.listdir(category_path):
                    file_path = os.path.join(category_path, file)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        text = f.read().strip()
                        data[split.lower()].append({"text": text, "label": category})
    return data

# Charger les données
print("Chargement des données...")
data = load_data(extract_dir)

# Convertir en DataFrame
train_df = pd.DataFrame(data["train"])
test_df = pd.DataFrame(data["test"])

# Aperçu des données
print("Aperçu des données d'entraînement :")
print(train_df.head())
print(f"Nombre de données d'entraînement : {len(train_df)}")
print(f"Nombre de données de test : {len(test_df)}")


Utilisation de l'appareil : cuda
Extraction des fichiers...
Extraction terminée.
Chargement des données...
Aperçu des données d'entraînement :
                                                text label
0  Body\nدبي:«الخليج» أعلن منظمو معرض ومسابقة «جل...  Tech
1  Body\nزار سمو الشيخ راشد بن سعود بن راشد المعل...  Tech
2  Body\nدبي: «الخليج» أعلنت كل من شركة الثريا لل...  Tech
3  Body\nوزعت اتصالات الجوائز على الدفعة الأولى م...  Tech
4  Body\nقدمت باناسونيك مجموعة من حلول العرض الجد...  Tech
Nombre de données d'entraînement : 99810
Nombre de données de test : 11090


In [15]:

# Mapper les labels (catégories) en indices numériques
categories = train_df["label"].unique()
label_mapping = {label: idx for idx, label in enumerate(categories)}

train_df["label"] = train_df["label"].map(label_mapping)
test_df["label"] = test_df["label"].map(label_mapping)

# Séparer les données en textes et labels
train_texts = train_df["text"].tolist()
train_labels = train_df["label"].tolist()

test_texts = test_df["text"].tolist()
test_labels = test_df["label"].tolist()

# Fractionner les données d'entraînement en sous-ensembles entraînement/validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=42
)

# Résumé des jeux de données
print(f"Nombre de textes d'entraînement : {len(train_texts)}")
print(f"Nombre de textes de validation : {len(val_texts)}")
print(f"Nombre de textes de test : {len(test_texts)}")

Nombre de textes d'entraînement : 89829
Nombre de textes de validation : 9981
Nombre de textes de test : 11090


2-Tokenisation avecle tokeniseur araBert:

On utilise le tokenizer de BERT pour convertir les textes en tokens exploitables par le modèle.

In [16]:
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

# Initialiser le tokenizer BERT
MODEL_NAME = "aubmindlab/bert-base-arabertv2"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Fonction pour tokeniser les textes
def tokenize_data(texts, labels, tokenizer, max_length=128):

    encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
    return encodings, torch.tensor(labels)

# Tokeniser les jeux de données d'entraînement, validation et test
train_encodings, train_labels = tokenize_data(train_texts, train_labels, tokenizer)
val_encodings, val_labels = tokenize_data(val_texts, val_labels, tokenizer)
test_encodings, test_labels = tokenize_data(test_texts, test_labels, tokenizer)

# Création d'un Dataset PyTorch
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Créer les datasets pour PyTorch
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# Création des DataLoaders
BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print("Données préparées pour PyTorch.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Données préparées pour PyTorch.


3-Fine-Tuning du Modèle araBERT :

On entraîne un modèle araBert pour la classification de texte.

entrainement du model AraBert:

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(categories))
model.to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).numpy()

    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./bert_standard_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Créer un Trainer pour simplifier l'entraînement
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle entraîné
model.save_pretrained("./bert_standard_model")
tokenizer.save_pretrained("./bert_standard_model")
print("Modèle BERT standard entraîné et sauvegardé.")

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-17-7f0dff0b416f>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.108200,0.133780,0.966436


Epoch,Training Loss,Validation Loss,Accuracy
1,0.108200,0.133780,0.966436
2,0.096200,0.144077,0.965434
3,0.095900,0.126969,0.971446


Evaluation du model


In [ ]:
# Fonction d'évaluation
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch["labels"].cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=categories)
    return accuracy, report

# Créer un DataLoader pour le test
test_loader = DataLoader(test_dataset, batch_size=16)

# Évaluer sur le jeu de test
test_accuracy, test_report = evaluate_model(model, test_loader)
print(f"Accuracy sur le test : {test_accuracy}")
print("Rapport de classification :")
print(test_report)

Implémentation de l’Attention Différentielle


In [ ]:
class DifferentialAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, lambda_init=0.8):
        super(DifferentialAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Projections pour Q, K, V
        self.qkv = nn.Linear(embed_dim, embed_dim * 3, bias=False)

        self.lambda_q1 = nn.Parameter(torch.randn(self.head_dim))
        self.lambda_k1 = nn.Parameter(torch.randn(self.head_dim))
        self.lambda_q2 = nn.Parameter(torch.randn(self.head_dim))
        self.lambda_k2 = nn.Parameter(torch.randn(self.head_dim))
        self.lambda_init = lambda_init

    def forward(self, x):
        batch_size, seq_length, embed_dim = x.size()
        qkv = self.qkv(x).chunk(3, dim=-1)
        q, k, v = qkv
        q1, q2 = q.chunk(2, dim=-1)
        k1, k2 = k.chunk(2, dim=-1)
        attn_1 = F.softmax(torch.matmul(q1, k1.transpose(-2, -1)) / self.head_dim**0.5, dim=-1)
        attn_2 = F.softmax(torch.matmul(q2, k2.transpose(-2, -1)) / self.head_dim**0.5, dim=-1)
        lambda_val = torch.exp(self.lambda_q1 @ self.lambda_k1) - torch.exp(self.lambda_q2 @ self.lambda_k2) + self.lambda_init
        diff_attn = attn_1 - lambda_val * attn_2
        context = torch.matmul(diff_attn, v)
        return context

Modification du Modèle AraBERT:

On remplace 50% des couches d’attention par l’implémentation DifferentialAttention

In [ ]:
from transformers.models.bert.modeling_bert import BertSelfAttention

class BertSelfAttentionWithDiff(BertSelfAttention):
    def __init__(self, config):
        super().__init__(config)
        self.diff_attention = DifferentialAttention(
            embed_dim=config.hidden_size,
            num_heads=config.num_attention_heads
        )

    def forward(self, hidden_states, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=False):
        context_layer = self.diff_attention(hidden_states)
        return context_layer

def modify_bert_for_diff_attention():
    from transformers import BertConfig, BertModel
    config = BertConfig.from_pretrained(MODEL_NAME)
    model = BertModel(config)
    num_layers = len(model.encoder.layer)
    for i in range(num_layers // 2):  # Remplacer 50 % des couches
        model.encoder.layer[i].attention.self = BertSelfAttentionWithDiff(config)
    return model

# Charger le modèle modifié
model = modify_bert_for_diff_attention()
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-Tuning et Évaluation du Modèle Modifié:

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# Reconfiguration du Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
print("Entraînement du modèle modifié avec attention différentielle...")
trainer.train()

# Évaluation finale
print("Évaluation du modèle modifié sur le test set :")
trainer.evaluate(test_dataset)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Entraînement du modèle modifié avec attention différentielle...


TypeError: BertModel.forward() got an unexpected keyword argument 'labels'

Evaluation du model entrainer:


In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            batch = {key: val.to("cuda" if torch.cuda.is_available() else "cpu") for key, val in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch["labels"].cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=categories)
    return accuracy, report

test_loader = DataLoader(test_dataset, batch_size=16)
test_accuracy_diff, test_report_diff = evaluate_model(model, test_loader)
print(f"Accuracy sur le test (BERT avec attention différentielle) : {test_accuracy_diff}")
print("Rapport de classification (BERT avec attention différentielle) :")
print(test_report_diff)